# QML Tutorial

by Anders S. Christensen.

This tutorial is a general introduction to kernel-ridge regression with Quantum Machine Learning (QML).

QML is freely available under the terms of the MIT open-source license. However, we ask that you properly cite the relevant, underlying work in your published work (provided below).

##### QML Citation: 
AS Christensen, FA Faber, B Huang, LA Bratholm, A Tkatchenko, KR Müller, OA von Lilienfeld (2017) "QML: A Python Toolkit for Quantum Machine Learning" https://github.com/qmlcode/qml

##### Theory Citation:
[Rupp et al, Phys Rev Letters, 2012](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.108.058301)

[Montavon et al, New J Phys, 2013](http://iopscience.iop.org/article/10.1088/1367-2630/15/9/095003/meta)


## Theory 
Regression model of some property, 
\begin{equation*}
y
\end{equation*}
for some system, 
\begin{equation*}
\widetilde{\mathbf{X}}
\end{equation*}
this could correspond to e.g. the atomization energy of a molecule:

\begin{equation*}
y\left(\widetilde{\mathbf{X}} \right) = \sum_i \alpha_i \  K\left( \widetilde{\mathbf{X}}, \mathbf{X}_i\right)
\end{equation*}

E.g. Using Gaussian kernel function with Frobenius norm:
\begin{equation*}
K_{ij} = K\left( \mathbf{X}_i, \mathbf{X}_j\right) = \exp\left( -\frac{\| \mathbf{X}_i - \mathbf{X}_j\|_2^2}{2\sigma^2}\right)
\end{equation*}

Regression coefficients are obtained through kernel matrix inversion and multiplication with reference labels
\begin{equation*}
\boldsymbol{\alpha} = (\mathbf{K} + \lambda \mathbf{I})^{-1} \mathbf{y}
\end{equation*}

## Load QML into Jupyter Notebook

In [1]:
from __future__ import print_function

import sys
print("Printing version info for help reporting bugs")
print("Python version:", sys.version)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

try:
    import qml
    from qml.kernels import gaussian_kernel
    from qml.kernels import laplacian_kernel
    from qml.math import cho_solve
    print("QML version:",qml.__version__)
except ImportError:
    print("Failed to find QML")
    print("Please follow instructions here: http://www.qmlcode.org/installation.html")

Printing version info for help reporting bugs
Python version: 3.6.5 (default, Apr 26 2018, 00:14:31) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-11)]
QML version: 0.4.0


## Tutorial exercises

If you got this notebook without the repo, please clone the following GIT repository to access the necessary scripts and QM7 dataset (atomization energies and relaxed geometries at PBE0/def2-TZVP level of theory) for ~7k GDB1-7 molecules. 
[Ruddigkeit et al, J Chem Inf Model, 2012](https://pubs.acs.org/doi/abs/10.1021/ci300415d)
[Rupp et al, Phys Rev Letters, 2012](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.108.058301)

git clone https://github.com/qmlcode/tutorial.git

Additionally, the repository contains Python3 scripts with the solutions to each exercise.

# Exercise 1: Representations

In this exercise we use QML to generate the Coulomb matrix and Bag of bonds (BoB) representations. [Montavon et al, New J Phys, 2013](http://iopscience.iop.org/article/10.1088/1367-2630/15/9/095003/meta)
In QML data can be parsed via the <font color=red>Compound</font> class, which stores data and generates representations in Numpy's ndarray format.
If you run the code below, you will read in the file <font color=red>qm7/0001.xyz</font> (a methane molecule) and generate a coulomb matrix representation (sorted by row-norm) and a BoB representation.

In [2]:
# Create the compound object mol from the file qm7/0001.xyz which happens to be methane
mol = qml.Compound(xyz="qm7/0001.xyz")

# Generate and print a coulomb matrix for compound with 5 atoms 
mol.generate_coulomb_matrix(size=5, sorting="row-norm")
print(mol.representation)

# Generate and print BoB bags for compound containing C and H
mol.generate_bob(asize={"C":2, "H":5})
print(mol.representation)

[36.8581052   5.50857022  0.5         5.50857007  0.56221501  0.5
  5.5085695   0.56221605  0.56221611  0.5         5.50856526  0.56221669
  0.56221777  0.56221405  0.5       ]
[36.8581052   0.          0.          5.50857022  5.50857007  5.5085695
  5.50856526  0.          0.          0.          0.          0.
  0.          0.5         0.5         0.5         0.5         0.
  0.56221777  0.56221669  0.56221611  0.56221605  0.56221501  0.56221405
  0.          0.          0.          0.        ]


The representations are simply stored as 1D-vectors.
Note the keyword <font color=red>size</font> which is the largest number of atoms in a molecule occurring in test or training set. 
Additionally, the coulomb matrix can take a sorting scheme as keyword, and the BoB representations requires the specifications of how many atoms of a certain type to make room for in the representations.

Lastly, you can print the following properties which is read from the XYZ file:

In [3]:
# Print other properties stored in the object
print(mol.coordinates)
print(mol.atomtypes)
print(mol.nuclear_charges)
print(mol.name)
print(mol.unit_cell)

[[ 1.041682 -0.0562   -0.071481]
 [ 2.130894 -0.056202 -0.071496]
 [ 0.678598  0.174941 -1.072044]
 [ 0.678613  0.694746  0.62898 ]
 [ 0.678614 -1.038285  0.228641]]
['C', 'H', 'H', 'H', 'H']
[6 1 1 1 1]
qm7/0001.xyz
None


# Exercise 2: Kernels

In this exercise we generate a Gaussian kernel matrix, 
\begin{equation*}
\mathbf{K}
\end{equation*}
using the representations
\begin{equation*}
\mathbf{X}
\end{equation*}
which are generated similarly to the example in the previous exercise:
\begin{equation*}
K_{ij} = \exp\left( -\frac{\| \mathbf{X}_i - \mathbf{X}_j\|_2^2}{2\sigma^2}\right)
\end{equation*}
QML supplies functions to generate the most basic kernels (E.g. Gaussian, Laplacian). In the exercise below, we calculate a Gaussian kernel for the QM7 dataset.
In order to save time you can import the entire QM7 dataset as  <font color=red>Compound</font> objects from the file  <font color=red>tutorial_data.py</font> found in the tutorial GitHub repository.

In [4]:
# Import QM7, already parsed to QML
from tutorial_data import compounds

# For every compound generate a coulomb matrix or BoB
for mol in compounds:

    mol.generate_coulomb_matrix(size=23, sorting="row-norm")
    # mol.generate_bob(size=23, asize={"O":3, "C":7, "N":3, "H":16, "S":1})

# Make a big 2D array with all the representations
X = np.array([mol.representation for mol in compounds])

# Print all representations
print(X)

# Run on only a subset of the first 1000 (for speed)
X = X[:1000]

# Define the kernel width
sigma = 1000.0

# K is also a Numpy array
K = gaussian_kernel(X, X, sigma)

# Print the kernel
print(K)

[[73.51669472 19.91549544 73.51669472 ...  0.          0.
   0.        ]
 [73.51669472 24.77884434 53.3587074  ...  0.          0.
   0.        ]
 [73.51669472 28.39419169 73.51669472 ...  0.          0.
   0.        ]
 ...
 [73.51669472 39.57492699 36.8581052  ...  0.          0.
   0.        ]
 [73.51669472 27.22906335 73.51669472 ...  0.          0.
   0.        ]
 [73.51669472 39.78475661 36.8581052  ...  0.          0.
   0.        ]]
[[1.         0.99795995 0.99864847 ... 0.99905645 0.99812809 0.99747646]
 [0.99795995 1.         0.99795162 ... 0.99828487 0.99972878 0.9976251 ]
 [0.99864847 0.99795162 1.         ... 0.99885073 0.9981792  0.99752914]
 ...
 [0.99905645 0.99828487 0.99885073 ... 1.         0.99857214 0.9987595 ]
 [0.99812809 0.99972878 0.9981792  ... 0.99857214 1.         0.99788663]
 [0.99747646 0.9976251  0.99752914 ... 0.9987595  0.99788663 1.        ]]


## Exercise 3: Regression

With the kernel matrix and representations sorted out in the previous two exercise, we can now solve the 

\begin{equation*}
\boldsymbol{\alpha}
\end{equation*} 

regression coefficients:

\begin{equation*}
\boldsymbol{\alpha} = (\mathbf{K} + \lambda \mathbf{I})^{-1} \mathbf{y}
\end{equation*}

One of the most efficient ways of solving this equation is using a Cholesky-decomposition.
QML includes a function named <font color=red>cho_solve()</font> to do this via the math module <font color=red>qml.math</font>.
In this step it is convenient to only use a subset of the full dataset as training data (see below).
The following builds on the code from the previous step.
To save time, you can import the PBE0/def2-TZVP atomization energies for the QM7 dataset from the file <font color=red>tutorial_data.py</font>.
This has been sorted to match the ordering of the representations generated in the previous exercise.
Extend your code from the previous step with the code below:


In [5]:
from tutorial_data import energy_pbe0

# Assign 1000 first molecules to the training set
X_training = X[:1000]
Y_training = energy_pbe0[:1000]

sigma = 4000.0
K = gaussian_kernel(X_training, X_training, sigma)
print(K)

# Add a small lambda to the diagonal of the kernel matrix
K[np.diag_indices_from(K)] += 1e-8

print(len(K))
print(len(Y_training))


# Use the built-in Cholesky-decomposition to solve
alpha = cho_solve(K, Y_training)

print(alpha)

[[1.         0.99987237 0.99991548 ... 0.999941   0.9998829  0.99984209]
 [0.99987237 1.         0.99987185 ... 0.99989272 0.99998305 0.9998514 ]
 [0.99991548 0.99987185 1.         ... 0.99992813 0.9998861  0.99984539]
 ...
 [0.999941   0.99989272 0.99992813 ... 1.         0.9999107  0.99992242]
 [0.9998829  0.99998305 0.9998861  ... 0.9999107  1.         0.99986778]
 [0.99984209 0.9998514  0.99984539 ... 0.99992242 0.99986778 1.        ]]
1000
1000
[ 2.27792067e+09 -1.02252571e+09 -5.08751603e+08  1.22295207e+09
 -6.57718637e+08  2.57277760e+09 -1.57445226e+08 -5.20879350e+08
  1.25864418e+09  1.59068718e+07 -1.61319348e+09 -5.47179964e+08
 -8.83818584e+08 -7.08245848e+08  4.76566462e+08 -1.19741800e+09
 -1.03916151e+09  1.22268511e+08 -1.43150882e+09 -1.44851064e+09
 -8.65609117e+08  2.19972771e+09 -9.37737176e+08  5.35442253e+08
 -3.31995869e+06 -1.44634843e+09 -2.03426603e+09 -2.18121529e+09
  1.80982079e+09 -1.52603827e+09  1.38844883e+09 -1.69434225e+09
 -6.85272902e+08  3.004096

## Exercise 4: Prediction

With the \begin{equation*}\boldsymbol{\alpha}\end{equation*} regression coefficients from the previous step, we have (successfully) trained the machine, and we are now ready to do predictions for other compounds.
This is done using the following equation:
    
\begin{equation*}
y\left(\widetilde{\mathbf{X}} \right) = \sum_i \alpha_i \  K\left( \widetilde{\mathbf{X}}, \mathbf{X}_i\right)
\end{equation*}

In this step we further divide the dataset into a training and a test set. Try using the last 1000 entries as test set.

In [6]:
# Assign 1000 last molecules to the test set
X_test = X[-1000:]
Y_test = energy_pbe0[-1000:]

# calculate a kernel matrix between test and training data, using the same sigma
Ks = gaussian_kernel(X_test, X_training, sigma)

# Make the predictions
Y_predicted = np.dot(Ks, alpha)

# Calculate mean-absolute-error (MAE):
print(np.mean(np.abs(Y_predicted - Y_test)))

254.2353172018013


## Exercise 5: Learning curves

Repeat the prediction from Exercise 2.4 with training set sizes of 1000, 2000, and 4000 molecules.

Note the MAE for every training size.

Plot a learning curve of the MAE versus the training set size.

Generate a learning curve for the Gaussian and Laplacian kernels, as well using the coulomb matrix and bag-of-bonds representations.

Which combination gives the best learning curve? Note you will have to adjust the kernel width (sigma) underway.

## Exercise 6: Delta learning

A powerful technique in machine learning is the delta learning approach. Instead of predicting the PBE0/def2-TZVP atomization energies, we shall try to predict the difference between DFTB3 (a semi-empirical quantum method) and PBE0 atomization energies.
Instead of importing the <font color=red>energy_pbe0</font> data, you can import the <font color=red>energy_delta</font> and use this instead

In [7]:
from tutorial_data import energy_delta

Y_training = energy_delta[:1000]
Y_test = energy_delta[-1000:]

Finally re-draw one of the learning curves from the previous exercise, and note how the prediction improves.